In [13]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [14]:
events = utils.get_events()
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


In [15]:
events.shape

(2341681, 26)

In [16]:
campaign_events = events[events['event']=='ad campaign hit'][['timestamp', 'event', 'person', 'url',
       'campaign_source', 'day', 'month']]
campaign_events['ads_accion'] = campaign_events['url'].str.split('/').str[1]
campaign_events['ads_marca'] = campaign_events['url'].str.split('/').str[2]
campaign_events['ads_modelo'] = campaign_events['url'].str.split('/').str[3]

In [17]:
## source
campaign_data = campaign_events[campaign_events['campaign_source'].notnull()]
campaign_data['campaign_source'].value_counts()

google              123354
criteo               24931
rtbhouse             23540
zanox                 7013
emblue                3653
afilio                1913
manifest              1365
FacebookAds           1301
voxus                  971
rakuten                843
bing                   839
datacrush              433
indexa                 313
buscape                271
Facebook               203
afiliado                92
blog                    88
mercadopago             74
yotpo                   54
FacebookSocial          12
onsite                  12
MARKETING SOCIAL         7
gizmodo                  4
Name: campaign_source, dtype: int64

In [18]:
features = events['person']
features = pd.concat([features, pd.get_dummies(events['campaign_source'], prefix='campaign_source')],axis=1)

In [19]:
features.shape

(2341681, 24)

In [22]:
features = features.groupby('person').sum()
features = features.reset_index()
features = features.fillna(0)
features.shape

(38829, 24)

In [23]:
features.to_csv('features_ad_campaing.csv',index=False)